# CET 324 Authentication System

__Notes__    
Used bcrypt for reasons given by: https://blog.cloudflare.com/keeping-passwords-safe-by-staying-up-to-date/    
Used AES for encryption of token. Create a class with a defined key for the AES encryption.    
Used Asymmetric encryption to encrypt the private shared key for Symmetric encryption

Now need to fix code so that token entered by user can be decrypted using the key from the file. System should read the decrypted token and check it is legitimate (i.e. correct signature) and is valid (today < expirydate)

In [129]:
from datetime import date
from dateutil.relativedelta import relativedelta
import hashlib, io, hmac
from cryptography.fernet import Fernet
import csv
import bcrypt

In [130]:
class User:
    def __init__(self, username, password, permission):
        self.username = username
        self.password = password
        self.permission = permission
        
    def requestAccessToken(self, system):
        accessToken = AccessToken(self, system)
        return accessToken.generateToken().decode('utf-8')
    
    def userAccountPage(self, system):
        print(self.username + "\'s Account Page")
        print("___________________")
        while 1: 
            print("1. Request token")
            print("0. Logout")
            
            user_command = int(input("Enter your command: "))
            
            #if else statement to carry out user command 
            if user_command == 1:
                new_token = self.requestAccessToken(system)
                print(new_token)
            elif user_command == 0:
                print("Logging Out")
                break
            else: 
                print("Invalid Command")

In [131]:
class AccessToken:
    def __init__(self, user, system):
        self.user = user
        self.system = system
        self.date = date.today()
        self.token = ''
        self.key = ''
        
    #function to read file 
    def read_file(self, path):
        file = open(path, 'r+')
        return file.read()
    
    def calcExpiryDate(self):
        #add 6 months to current date to calculate expiry date 
        return self.date + relativedelta(months=6)
    
    def generateToken(self):        
        #generate a token containing username, permission, date, expiry date and digital signature 
        self.token = self.user.username + "!£$" + self.user.permission + "!£$" + str(self.date) + "!£$" + str(self.calcExpiryDate()) + "!£$" + self.system.systemName
        self.token = self.token + self.createDigitalSignature().decode('utf-8')
        return self.encryptToken(self.token)
        
    def createDigitalSignature(self):
        #digital signature uses the username and system name with added special characters
        signature = self.user.username + '!£$%^&*' + self.system.systemName
        #hash the signature 
        return self.hashString(self.token)
    
    def hashString(self, plainString): 
        # convert string to array of bytes 
        bytes = plainString.encode('utf-8')
        #generate salt 
        salt = bcrypt.gensalt()
        #hash and salt password 
        hashString = bcrypt.hashpw(bytes, salt)
        return hashString
    
    def importKey(self, path):
        self.key = self.read_file(path)
    
    def encryptToken(self, token):
        #create the key 
        self.importKey(self.system.systemKeyPath)
        key = Fernet(self.key)
        #encrypt token using the key 
        encrypted_token = key.encrypt(bytes(token, 'utf-8'))
        return encrypted_token

In [132]:
class System:    
    #set default permission to 'u' (user) for creating a new account
    DEFAULT_PERMISSION = "u"
   
    def __init__(self, systemName):
        # replace spaces from system name 
        self.systemName = systemName.replace(' ', '_')
        self.systemKeyPath = "key-" + self.systemName + ".txt"
        
    #method to read from a file 
    def read_file(self, path):
        file = open(path, 'r+')
        return file.read()
    
    #method to write to a file 
    def write_file(self, path, text): 
        file = open(path, 'w')
        try:
            return file.write(text.decode('utf-8'))
        except:
            return file.write(text)
        
    #generate a key for the system and save in txt file to be used by authentication system    
    def createSystemKey(self):
        key = Fernet.generate_key()
        self.write_file(self.systemKeyPath, key)      
        
    def login(self, username, password): 
        with open(self.systemName + 'users.csv', newline='') as userscsv:
            fieldnames = ['username', 'password', 'permission', 'token', 'key']
            reader = csv.DictReader(userscsv, fieldnames=fieldnames)
            for row in reader:
                if row['username'] == username:
                    #convert password to bytes 
                    enteredPassword = password.encode('utf-8')
                    
                    savedPassword = row['password']
                
                    #check password 
                    if bcrypt.checkpw(enteredPassword, savedPassword.encode('utf-8')):
                        return User(row['username'], row['password'], row['permission'])
                    
            #return null if there is no match
            return None
    
            
    def hashpassword(self, plainPassword):
        #convert password to array of bytes 
        bytes = plainPassword.encode('utf-8')
        
        #generate salt 
        salt = bcrypt.gensalt()
        
        #hash and salt password 
        hashpassword = bcrypt.hashpw(bytes, salt)
        
        return hashpassword
    
        #function to decrypt text 
    def decrypt_text(self, key, text):
        # get the key 
        key = Fernet(key)
        # decrypt text using the key 
        decrypted_text = key.decrypt(text)
        return decrypted_text
        
    def loginWithToken(self, token):
        key = self.read_file(self.systemKeyPath)
        token = self.decrypt_text(key, token)
        self.checkTokenIsValid(token)
        
    def checkTokenIsValid(self, token):
        username = token.split('!£$')[0]
        print(username)
        
    
    def createAccount(self, user):
        with open(self.systemName + 'users.csv', 'a', newline='') as userscsv:
            fieldnames = ['username', 'password', 'permission', 'token']
            writer = csv.DictWriter(userscsv, delimiter=',', quotechar='|', 
                                   quoting=csv.QUOTE_MINIMAL, fieldnames=fieldnames)
            hashedpw = self.hashpassword(user.password)
        
            writer.writerow({'username': user.username, 'password': hashedpw.decode('utf-8'), 'permission': user.permission})

    def read_csv(self):
        with open(self.systemName + 'users.csv', newline='') as userscsv:
            fieldnames = ['username', 'password']
            reader = csv.DictReader(userscsv, fieldnames=fieldnames)
            for row in reader:
                print("", row['username'], row['password'], row['permission'])
            
    def display_main_menu(self):
        while 1: 
            print(self.systemName)
            print("___________________")
            print("Main Menu")
            print("1. Login")
            print("2. Create Account")
            print("3. Login With Access Token")
            print("0. Exit")
            
            user_command = int(input("Enter your command: "))
            
            #if else statement to carry out user command 
            if user_command == 1:
                username = input("Enter Username: ")
                password = input("Enter password: ")
                user = self.login(username,  password)
                if user is None:
                    print("Incorrect username or password")                    
                else:
                    user.userAccountPage(self)
            elif user_command == 2:
                username = input("Enter new Username: ")
                password = input("Enter new password: ")
                self.createAccount(User(username, password, self.DEFAULT_PERMISSION))
            elif user_command == 3:
                accessToken = input("Enter Access Token: ")
                self.loginWithToken(accessToken)
            elif user_command == 0:
                print("Exiting")
                break
            else: 
                print("Invalid Command")

In [133]:
system = System("Super Secure System")
system.createSystemKey()

system.display_main_menu()

Super_Secure_System
___________________
Main Menu
1. Login
2. Create Account
3. Login With Access Token
0. Exit
Enter your command: 1
Enter Username: user
Enter password: password
user's Account Page
___________________
1. Request token
0. Logout
Enter your command: 1
gAAAAABmM6w2po-vG6IAZQSR9ch3r8asuDcPrXTBpS4ZtvTQNdx53kPbBe24-wQWepNb86sNXbRS_9qzw2CbIxZf4P81Z_LO5ku2gL7JfNYGvnDXQv57z31D7R0eG8iHE42ptRXK_DyB34z_V5ZkeJSpz1BHLPaf5eQf6L3ELQaXsQ9EsqafMS9JLJQVd-jxkCjmJsfJIvtsC1yMh0_FiqRH6DDjN6iGCumCuTW3Lqv3k6fE7kcp0Fc=
1. Request token
0. Logout
Enter your command: 0
Logging Out
Super_Secure_System
___________________
Main Menu
1. Login
2. Create Account
3. Login With Access Token
0. Exit
Enter your command: 3
Enter Access Token: gAAAAABmM6w2po-vG6IAZQSR9ch3r8asuDcPrXTBpS4ZtvTQNdx53kPbBe24-wQWepNb86sNXbRS_9qzw2CbIxZf4P81Z_LO5ku2gL7JfNYGvnDXQv57z31D7R0eG8iHE42ptRXK_DyB34z_V5ZkeJSpz1BHLPaf5eQf6L3ELQaXsQ9EsqafMS9JLJQVd-jxkCjmJsfJIvtsC1yMh0_FiqRH6DDjN6iGCumCuTW3Lqv3k6fE7kcp0Fc=


TypeError: a bytes-like object is required, not 'str'